## Spotify Web API data


In [8]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [10]:
from dotenv import load_dotenv
import os
import base64
from requests import post
import json

load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
REDIRECT_URI = "https://localhost/"


def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")
    
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}
token = get_token()

sp_oauth = SpotifyOAuth(client_id=client_id, client_secret=client_secret, redirect_uri = REDIRECT_URI, scope = "user-library-read")

access_token = sp_oauth.get_access_token()



C:\Users\justi\AppData\Local\Temp\ipykernel_37320\3165804697.py:36: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  access_token = sp_oauth.get_access_token()


{'access_token': 'BQBb5WQ9APa4IJa8tImPdiaZu_RqymAh_aGJB6QswpcqjyIJE8NPfgvJVCanGIljRcDYxVmekAbadKD1xOVaaPCBRPRTiiw7a4-4mryMGYxwsIKsSEmROLK5hRa39VgXEBuXClXL61ZDR6zBXJADQNbTb9vFbwTgx2Fo22z8iDQNMpvKk6-hLmv5mlgldX1K2mT5N0pIStUpX-A', 'token_type': 'Bearer', 'expires_in': 3600, 'refresh_token': 'AQBs6dwsbc67u4A7YiW-5YqhsPMtAr-pY57Hdf8R5nHpwmGf-zKyKYVZnotWTjxfKGiyivSlGEKkoZ7WA_FwlriAaY3zDyxgv5k71pX9AZTceAR4Uc5VZ6NZ2geF5_D65pM', 'scope': 'user-library-read', 'expires_at': 1696097565}


In [12]:
sp = spotipy.Spotify(auth_manager=sp_oauth)